In [5]:
!pip install spacy sklearn rowordnet
!python -m spacy download ro_core_news_lg
!pip install bs4
!pip install sparknlp pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.5/568.5 MB 2.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ro_core_news_lg')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Functia de parsare a fiserului xml**

---



In [82]:
from bs4 import BeautifulSoup
import spacy




def parse_xml(path):
  with open(path, encoding='utf8') as fp:
    file_p = fp.read()
    soup = BeautifulSoup(file_p, 'html.parser')
  data = soup.find_all('instance')
  parsed_data = {}
  for sense in data:
    new_data = []
    # pentru fiecare sens, extragem id si contextele 
    contexts = sense.find_all('context')
    senseId = sense.attrs["id"][10:]
    if len(senseId)<=1:
      break
    word = sense.attrs["word"]
    for context in contexts:
      try:

        sentence = context.decode_contents().strip('\n')
        a, *b = sentence.split('<head>')
        b, *c = ''.join(b).split('</head>')
        sentence = a + b + ''.join(c)
        sentence= ' '.join(sentence.split())
        
        new_data.append((sentence, senseId))
          
      except:
        continue
    if word =='':
      continue
    # grupam sensurile pe cuvinte
    if word in parsed_data:
      parsed_data[word].extend(new_data)
    else:
      parsed_data[word] = new_data
     
  return parsed_data



prep_data = parse_xml("/content/drive/MyDrive/wsd/data_wsd/toate.xml")
print(prep_data.keys())




dict_keys(['lac', 'lin', 'mare', 'masa', 'opus', 'pană', 'păr', 'para', 'nouă', 'bancă', 'barem', 'broască', 'călca', 'car', 'colonie', 'consola', 'casa', 'acaju', 'accident', 'acru', 'afină', 'agățătoare', 'alpaca', 'abate', 'anticar', 'antrenor', 'arici', 'vin', 'veselă', 'vâna', 'umbrele', 'turele', 'torturi', 'toc', 'sare', 'republica', 'paria', 'somn', 'post', 'sol', 'poartă', 'ochi', 'mobilă', 'mie', 'formă', 'corn', 'coș', 'fata', 'fila', 'folie', 'haină', 'imobil', 'curele', 'copii'])


**Functia de splituire, care imparte datele in date de antrenare si date de test**

---



In [83]:
import random
def split_data(data):
  train_data = {}
  test_data = {}
  for word, vector in data.items():
    if(len({item[1] for item in vector})<2):
      continue
    senses = {}
    for tup in vector:
      context, sense = tup
      if sense not in senses:
        senses[sense] = []
      senses[sense].append(tup)

    train_data[word] = []
    test_data[word] = []


    for sense in senses:
      vec = senses[sense]
      l = len(vec)
      marg = 2*l//3
      if l < 3:
        pass

      else:
        random.shuffle(vec)
        ex_train = vec[:marg]
        ex_test = vec[marg:]
        train_data[word].extend(ex_train)
        test_data[word].extend(ex_test)
  return train_data, test_data

train_data, test_data = split_data(prep_data)
print(train_data.keys())
print(prep_data.keys())
print(train_data["lac"])
print(test_data["lac"])

dict_keys(['lac', 'lin', 'mare', 'masa', 'opus', 'pană', 'păr', 'para', 'nouă', 'bancă', 'broască', 'călca', 'car', 'colonie', 'consola', 'casa', 'accident', 'acru', 'alpaca', 'abate', 'anticar', 'vin', 'veselă', 'vâna', 'umbrele', 'turele', 'torturi', 'toc', 'sare', 'republica', 'paria', 'somn', 'post', 'sol', 'poartă', 'ochi', 'mobilă', 'mie', 'formă', 'corn', 'fata', 'imobil'])
dict_keys(['lac', 'lin', 'mare', 'masa', 'opus', 'pană', 'păr', 'para', 'nouă', 'bancă', 'barem', 'broască', 'călca', 'car', 'colonie', 'consola', 'casa', 'acaju', 'accident', 'acru', 'afină', 'agățătoare', 'alpaca', 'abate', 'anticar', 'antrenor', 'arici', 'vin', 'veselă', 'vâna', 'umbrele', 'turele', 'torturi', 'toc', 'sare', 'republica', 'paria', 'somn', 'post', 'sol', 'poartă', 'ochi', 'mobilă', 'mie', 'formă', 'corn', 'coș', 'fata', 'fila', 'folie', 'haină', 'imobil', 'curele', 'copii'])
[('mai bun și grâul a crescut mai des și mai viguros. Acolo era toată speranța de recoltă pentru anul acesta. Avea de 

**Functiile responsabile cu feature extraction**

In [102]:
import nltk
import spacy
from sklearn.feature_extraction import DictVectorizer
import rowordnet as rwn
from nltk.classify import MaxentClassifier

vec = DictVectorizer()
wn = rwn.RoWordNet()
nlp = spacy.load("ro_core_news_lg")

# extragerea hipernimelor
def extract_hypernymes(word):
    hyper = []
    synset_ids = wn.synsets(literal=word.lemma_, pos=rwn.synset.Synset.Pos.NOUN)
    if len(synset_ids) > 0:
        c = wn.synset_to_hypernym_root(synset_ids[0])
        for c1 in c:
            if len(wn(c1).literals):
                hyper.append(wn(c1).literals[0])
    hyper_dict = {}
    l = 5 if len(hyper)>2 else len(hyper)
    for i in range(len(hyper)):
      hyper_dict["hyper"+str(i)] = hyper[i]

    return hyper_dict

# functie care creaza pentru fiecare cuvant un dictionar de trasaturi, continand 
# informatii legate de morfologia (lemma, morpho), sintaxa(pos, dep) si
# sensul(hyper) cuvantului
def extract_feat_for_word(word, rel):
    hyper = {}
    if word.pos_ == "NOUN":
        hyper = extract_hypernymes(word)
    morph = word.morph.to_dict()
    d = {
        ("text"+rel): word.text,
        "pos"+rel: word.pos_,
        "lemma"+rel: word.lemma_,
        "dep"+rel: word.dep_,
         "ner"+rel: word.ent_type_,
    }
    d.update(morph)
    d.update(hyper)
    return d

# pentru a cuprind si informatia contextuala, se creaza dictionar de features nu doar pt cuvantul target cat si pentru cuvintele din propozitie 
# cu care se afla in relatie sintactica
def create_features_vector(word, context):
    text = nlp(context)
    vect = {}
    for w in text:
      if w.lemma_ == word:
            if w.head != w:
              vect.update(extract_feat_for_word(w.head, "head"))
              for ind, siblings in enumerate(w.head.children):
                vect.update(extract_feat_for_word(siblings, "0_"+str(ind) if w != siblings else "main"))
            else:
              vect.update(extract_feat_for_word(w, "main"))
            for chil in w.children:
              vect.update(extract_feat_for_word(chil, "children"))
        
    return vect

# functie care aplica tehnicile de feature extraction pe datele de intrare
def prepare_data(data):
  new_data_per_words = {}

  for word, instances in data.items():
    sense = word
    new_data = []
    
    for i in instances:
      new_data.append((create_features_vector(sense,i[0]), i[1]))
    new_data_per_words[word] = new_data


  return new_data_per_words

a = prepare_data(train_data)
b = prepare_data(test_data)

print(a.keys())
print(b['lac'])

dict_keys(['lac', 'lin', 'mare', 'masa', 'opus', 'pană', 'păr', 'para', 'nouă', 'bancă', 'broască', 'călca', 'car', 'colonie', 'consola', 'casa', 'accident', 'acru', 'alpaca', 'abate', 'anticar', 'vin', 'veselă', 'vâna', 'umbrele', 'turele', 'torturi', 'toc', 'sare', 'republica', 'paria', 'somn', 'post', 'sol', 'poartă', 'ochi', 'mobilă', 'mie', 'formă', 'corn', 'fata', 'imobil'])
[({'texthead': 'sud', 'poshead': 'NOUN', 'lemmahead': 'sud', 'dephead': 'ROOT', 'nerhead': '', 'Definite': 'Ind', 'Gender': 'Masc', 'Number': 'Sing', 'hyper0': 'lac', 'hyper1': 'cardinal', 'hyper2': 'direcție', 'hyper3': 'așezare', 'hyper4': 'legătură', 'hyper5': 'abstractizare', 'hyper6': 'entitate', 'text0_0': 'Și', 'pos0_0': 'CCONJ', 'lemma0_0': 'și', 'dep0_0': 'cc', 'ner0_0': '', 'Polarity': 'Pos', 'text0_1': 'mai', 'pos0_1': 'ADV', 'lemma0_1': 'mai', 'dep0_1': 'advmod', 'ner0_1': '', 'text0_2': 'spre', 'pos0_2': 'ADP', 'lemma0_2': 'spre', 'dep0_2': 'case', 'ner0_2': '', 'AdpType': 'Prep', 'Case': 'Acc,No

**Crearea si antrenarea a cate un model de clasificare pentru fiecare cuvant target din datele de intrare**

In [99]:
import sklearn

algorithm = nltk.classify.MaxentClassifier.ALGORITHMS[0]
classifiers = {}

for w in a.keys():
  print(w)
  classifier = nltk.MaxentClassifier.train(a[w], algorithm, max_iter=50)
  classifiers[w] = classifier


lac
  ==> Training (50 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -1.38629        0.323
             2          -1.16265        0.790
             3          -0.99692        0.839
             4          -0.87220        0.887
             5          -0.77552        0.903
             6          -0.69840        0.903
             7          -0.63539        0.919
             8          -0.58286        0.935
             9          -0.53836        0.952
            10          -0.50014        0.952
            11          -0.46695        0.968
            12          -0.43783        0.968
            13          -0.41209        0.968
            14          -0.38915        0.968
            15          -0.36859        0.984
            16          -0.35006        0.984
            17          -0.33326        1.000
            18          -0.31796        1.000
            19          -0.30397        1.00

**Evaluarea modelelor de clasificare si realizarea unor statistici pe rezultate**

In [100]:
precision= {}
recall = {}
accuracy = {}
for w in b.keys():
  output_labels = classifiers[w].classify_many([x[0] for x in b[w]])
  metrics = sklearn.metrics.classification_report( output_labels, [x[1] for x in b[w]],output_dict=True, zero_division=0)
  l = [label for label in metrics.keys()  if type(metrics[label])==type(dict()) ]
  precision[w] = sum([metrics[label]['precision'] for label in l  ])/len(l)
  recall[w] = sum([metrics[label]['recall'] for label in l  ])/len(l)
  accuracy[w] = metrics['accuracy']

def min_score(d, metric):
  min_v = min(d.values())
  it = []
  for k,v in d.items():
    if v==min_v:
      it.append((k,v))
  print("Words with the worst ",metric,":",it)

def max_score(d, metric):
  max_v = max(d.values())
  it = []
  for k,v in d.items():
    if v==max_v:
      it.append((k,v))
  print("Words with the best ",metric,":",it)

def avg(d, metric):
  av = sum(d.values())/len(d)
  print(metric,":",av)

min_score(precision,"precision")
max_score(precision,"precision")
avg(precision,"precision")

min_score(recall,"recall")
max_score(recall,"recall")
avg(recall,"recall")

min_score(accuracy,"accuracy")
max_score(accuracy,"accuracy")
avg(accuracy,"accuracy")

Words with the worst  precision : [('nouă', 0.2125)]
Words with the best  precision : [('alpaca', 1.0), ('vâna', 1.0), ('turele', 1.0), ('mobilă', 1.0), ('corn', 1.0), ('imobil', 1.0)]
precision : 0.6143903507193198
Words with the worst  recall : [('nouă', 0.02361111111111111)]
Words with the best  recall : [('alpaca', 1.0), ('vâna', 1.0), ('turele', 1.0), ('mobilă', 1.0), ('corn', 1.0), ('imobil', 1.0)]
recall : 0.5021758878538465
Words with the worst  accuracy : [('nouă', 0.1111111111111111)]
Words with the best  accuracy : [('alpaca', 1.0), ('vâna', 1.0), ('turele', 1.0), ('mobilă', 1.0), ('corn', 1.0), ('imobil', 1.0)]
accuracy : 0.5981597417521789
